In [1]:
from copy import copy

In [26]:
VAR_INDEX = 0

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args 
        self.tree_args = []
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
    def __lt__(self, other):
        if type(self) == Var and type(other) == Var:
            return self.name < other.name
        if type(self) == Var and (type(other) == Conjunction or type(other) == Disjunction):
            return self.name <= other.args[0].name
        if  (type(self) == Conjunction or type(self) == Disjunction) and type(other) == Var:
            return self.args[0].name < other.name
        if  (type(self) == Conjunction and type(other) == Conjunction) or (type(self) == Disjunction and type(other) == Disjunction):
            self_names = []
            other_names = []
            for arg in self.args:
                self_names.append(arg.name)
            for arg in other.args:
                other_names.append(arg.name)
            print(self_names, other_names)
            return self_names < other_names
        
        
# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)
        

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name, specific_negated=False):
        super().__init__(None, negated=specific_negated)        
        global VAR_INDEX
        self.index = VAR_INDEX
        VAR_INDEX += 1
        self.name = name

In [3]:
def remove_imp_eq(formula):
    if type(formula) == Var:
        return formula
    
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1])
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0]) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Equivalence:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1]) + \
               remove_imp_eq(-(formula.args[0])) * remove_imp_eq(-(formula.args[1]))
    expr.negated = formula.negated
    return expr

def de_morgan(formula):
    if type(formula) == Var:
        return formula
    if type(formula) == Disjunction :
        if formula.negated:
            return de_morgan(-formula.args[0]) * de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) + de_morgan(formula.args[1])
    if type(formula) == Conjunction:
        if formula.negated:
            return de_morgan(-formula.args[0]) + de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) * de_morgan(formula.args[1])

def flatten_list(l):
    flat_list = []
    for list_elem in l:
        if type(list_elem) == list:
            for item in list_elem:
                flat_list.append(item)
        else:
            flat_list.append(list_elem)
    return flat_list

def remove_associativity(formula):
    if type(formula) != Var: 
        for i, arg in enumerate(formula.args):
            if (type(formula) == Disjunction and type(formula.args[i]) == Disjunction) or \
            (type(formula) == Conjunction and type(formula.args[i]) == Conjunction):
                formula.args[i] = formula.args[i].args
                formula.args = flatten_list(formula.args)
                remove_associativity(formula)  
        for i, arg in enumerate(formula.args):
            remove_associativity(arg)

In [4]:
def sort_leaves(formula):
    for i, arg in enumerate(formula.args):
        if type(arg) != Var:
            sort_leaves(arg)        
    for i, arg in enumerate(formula.args):
        if type(arg) != Var:
            return
    formula.args.sort()

In [64]:
def is_flatten(lst):
    for elem in lst:
        if type(lst) == list:
            return False
    return True

In [100]:
def get_tree_args(formula):
    if type(formula) != Var:
        for i, arg in enumerate(formula.args):
            get_tree_args(arg)
            if type(arg) == Conjunction or type(arg) == Disjunction:
                for j, inner_arg in enumerate(arg.args):
                    if type(inner_arg) == Var:
                        inner_arg.tree_args = inner_arg.name
                        arg.tree_args += [inner_arg.name]
                    else:
                        arg.tree_args += [inner_arg.tree_args]
                        
def get_root_tree(formula):
    for arg in formula.args:
        formula.tree_args.append(arg.tree_args)
        
def get_formula_tree(formula):
    get_tree_args(formula)
    get_root_tree(formula)

In [79]:
def sort_nodes(formula):
    if is_flatten(formula.tree_arg):
        return sorted(formula.tree_arg)

SyntaxError: invalid syntax (<ipython-input-79-9bbfef43011d>, line 2)

In [106]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')
e = Var('e')
g = Var('g')
a,b,c,d,e

(<__main__.Var at 0x22a6727d2e8>,
 <__main__.Var at 0x22a6727d6d8>)

In [107]:
f = (d+a+e*(d+a)*((g*g) + a)) * (a*g + c*c) * e * (b + a) 
remove_associativity(f)
sort_leaves(f)
get_formula_tree(f)

In [108]:
f.tree_args

[['d', 'a', ['e', ['a', 'd'], [['g', 'g'], 'a']]],
 [['a', 'g'], ['c', 'c']],
 'e',
 ['a', 'b']]

In [104]:
f.args[1].tree_args

[['a', 'g'], ['c', 'c']]

In [105]:
f.args[0].args[2].tree_args

['e', ['a', 'd'], [['g', 'g'], 'a']]